In [ ]:
sub_questions = [
    "What type of data structure is used here?",
    "Is candidates approach correct for answer?",
    "Is candidate using brute force approach here?",
    "Is candidate using pointers in answer?"
]

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from langchain_groq import ChatGroq
from typing import Annotated, TypedDict, Optional
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from IPython.display import Image, display
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel
from langchain_core.tools import tool
from langchain_core.messages import (
    BaseMessage,
    HumanMessage,
    ToolMessage,
)

In [ ]:
llm = ChatGroq(model="llama-3.2-90b-text-preview")

In [ ]:
class Candidate(BaseModel):
    name: str
    job_role: str

In [4]:
def create_agent(llm, tools, system_message: str):
    """Create an agent."""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful AI assistant, collaborating with other assistants."
                " Use the provided tools to progress towards answering the question."
                " If you are unable to fully answer, that's OK, another assistant with different tools "
                " will help where you left off. Execute what you can to make progress."
                " If you or any of the other assistants have the final answer or deliverable,"
                " prefix your response with FINAL ANSWER so the team knows to stop."
                " You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
    return prompt | llm.bind_tools(tools)

In [ ]:
class AgentState(TypedDict):
    candidate_name: Optional[str] = None
    candidate_role: Optional[str] = None
    current_question: Optional[str] = None
    current_answer: Optional[str] = None
    current_evaluation: Optional[str] = None
    history: Optional[str] = None

In [ ]:
def greet_user(state: AgentState):
    intro = "Hi, I am Noha, can you please introduce yourself and what you do?"
    print("Noha: ", intro)

    prompt_extraction = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are an expert extraction algorithm. "
                "Only extract relevant information from the text. "
                "If you do not know the value of an attribute asked to extract, "
                "return null for the attribute's value.",
            ),
            ("human", "{text}"),
        ]
    )
    user_input = input("User: ")
    runnable = prompt_extraction | llm.with_structured_output(schema=Candidate)
    candidate = runnable.invoke({'text': user_input})
    return {"candidate_name":candidate.name, "candidate_role":candidate.job_role}

In [ ]:
@tool
def get_main_question() -> dict:
    main_question = "Can you tell me how we can reverse words in string 'I am a boy'?"
    ideal_answer = '''#### Strategy
    Two Pointers.

    #### Explanation
    First, the algorithm splits the words of the sentence from a string into an array.\
    The algorithm also initializes low and high pointers as the first and last indexes of the array.\
    Then, the algorithm iterates the array.

    For each iteration, the algorithm swaps the words at the low and high pointers,\
    increments and decrements the low and high pointers respectively,\
    and repeats until the low pointer is greater than the high pointer.

    The result is all words of the sentence reversed in the array.\
    The words are joined as a string again with a single space separating the words, and the string is returned.

    ##### Time Complexity
    The algorithm updates the low and high pointers a number of times proportional to the number of words. \
    Therefore, the time complexity of the algorithm is O(n), where n is the number of words.

    ##### Space Complexity
    The algorithm splits the words from a sentence into an array. The array is auxiliary space and has length equal to the number of words.\
    Therefore, the auxiliary space complexity of the algorithm is O(n).'''
    return {"question": main_question, "answer": ideal_answer}


@tool
def get_sub_question(iteration: Annotated[int, ""]):
    sub_questions = [
        "What type of data structure is used here?",
        "Is candidates approach correct for answer?",
        "Is candidate using brute force approach here?",
        "Is candidate using pointers in answer?"
    ]
    return sub_questions[iteration]

In [ ]:
agent_1 = create_agent(
    llm=llm,
    tools=[get_main_question, get_sub_question],
    system_message="You will ask question to user and evaluate the answer given."
)

In [ ]:
def agent_1():
    return {}